In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Open Data Dashboard

QA script for dashboard_v3.py

In [1]:
import requests
import json
from io import StringIO

import os
import gspread

import pandas as pd
import numpy as np
from datetime import date, datetime

import warnings
warnings.filterwarnings('ignore')

In [72]:
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials

In [3]:
# pd.options.display.float_format = '{:.0f}'.format

In [4]:
# delete in production code
# from importlib import reload
# reload(credentials)

In [2]:
#### Socrata and Google Sheets credentials are associated with modanycga@gmail.com

#### SETTING UP SOCRATA ####

socrata_url = "https://data.cityofnewyork.us/resource/"

def call_socrata_api(uid, limit=100000):
    """
    Calls Soctata API to exctract a dataset based on its id

    Args:
        uid: str, Socrata id for the dataset to pull
    Returns:
        pandas df of the dataset
    """

    num_records = f"$limit={limit}"

    r = requests.get(socrata_url + uid + '.json?' + num_records)
    if r.status_code != 200:
        raise Exception('Error getting data')
    asset_df = pd.read_json(StringIO(json.dumps(r.json())))

    return asset_df

In [ ]:
#### SETTING UP GOOGLE SPREADSHEETS ####

# getting google service account credentials from 
# GitHub secrets and writing them to a file

# google_credential = os.getenv('GS_CREDENTIALS')
# home_path = os.getenv('HOME')
# creds_location = os.path.join(home_path,'service_account.json')

# with open(creds_location, 'w') as f:
#     f.write(google_credential)

In [4]:
## Google Spreadsheet key from the URL
## DEV:
gs_name = "ODD_DEV_Source_File"
## PROD:
# gs_name = "ODD_V3_Source_File"

gc = gspread.service_account(filename='/media/sf_VBUM_FCTF/ODD/dashboard/Jupyter_and_Google_Sheets-12b039cdb296.json')
sh = gc.open(gs_name)

def gs_upload(wks_name, df):
    worksheet = sh.worksheet(wks_name)
    worksheet.update([df.columns.values.tolist()] +\
               df.values.tolist())

In [81]:
scope = ['https://spreadsheets.google.com/feeds']
gs_creds = ServiceAccountCredentials.from_json_keyfile_name('/media/sf_VBUM_FCTF/ODD/dashboard/Jupyter_and_Google_Sheets-12b039cdb296.json', scope)

gs_key = '1uTuneWixsOlm5Cq8uVUzedJCM3jqNWxZ_HOkM5zAljU'
gs_name = "/ODD_DEV_Source_File"

def gs_upload(df, wks_name):
    """
    Uploads df to Google Spreadsheets
    
    Args:
        gs_key: str, spreadhsheet key
        df: pandas dataframe to upload
        wks_name: str, worksheet name
    """
    d2g.upload(
        df=df,
        gfile=gs_key, 
        wks_name=wks_name, 
        row_names=False,
        credentials=gs_creds
    )

In [5]:
#### Step 1. Load Published Data Asset Inventory

# Local Law 251 of 2017: Published Data Asset Inventory
# https://data.cityofnewyork.us/City-Government/Local-Law-251-of-2017-Published-Data-Asset-Invento/5tqd-u88y
public_df = call_socrata_api('5tqd-u88y')
public_df.shape

(3298, 24)

In [6]:
list(public_df)

['datasetinformation_agency',
 'uid',
 'name',
 'description',
 'type',
 'category',
 'legislativecompliance_datasetfromtheopendataplan',
 'url',
 'update_datemadepublic',
 'update_updatefrequency',
 'last_data_updated_date',
 'legislativecompliance_candatasetfeasiblybeautomated',
 'update_automation',
 'legislativecompliance_hasdatadictionary',
 'legislativecompliance_containsaddress',
 'legislativecompliance_geocoded',
 'legislativecompliance_existsexternallyll1102015',
 'legislativecompliance_externalfrequencyll1102015',
 'legislativecompliance_removedrecords',
 'visits',
 'derived_view',
 'row_count',
 'column_count',
 'parent_uid']

In [7]:
public_cols = [
    'datasetinformation_agency',
    'name',
    'uid',
    'url',
    'update_datemadepublic',
    'update_automation',
    'update_updatefrequency',
    'last_data_updated_date',
    'type',
    'row_count',
    'derived_view',
    'parent_uid'
]
public_df = public_df[public_cols]
public_df.shape

(3298, 12)

In [8]:
#### Step 2. Get dates of the data updates

# get the dates each of datasets has been updated
dates_df = public_df[public_df.uid.isin(['5tqd-u88y','qj2z-ibhs'])]\
                [['uid', 'last_data_updated_date']]
dates_df['last_data_updated_date'] = pd.to_datetime(dates_df.last_data_updated_date, 
                                                     errors='coerce')\
                                            .dt.strftime("%Y-%m-%d")

today_df = pd.DataFrame({'uid':['NA'],
                         'last_data_updated_date':[date.today().strftime("%Y-%m-%d")],
                         'Source':['1. Dashboard']})
dates_df.loc[dates_df.uid=='5tqd-u88y','Source'] = '2. Published Asset Inventory'
dates_df.loc[dates_df.uid=='qj2z-ibhs','Source'] = '3. Open Plan Tracker'
dates_df = dates_df.append(today_df)
dates_df.reset_index(inplace=True, drop=True)
dates_df = dates_df[['Source', 'last_data_updated_date']]
dates_df.rename(columns={'last_data_updated_date':'Updated on'},inplace=True)
dates_df

,Source,Updated on
0,3. Open Plan Tracker,2021-07-02
1,2. Published Asset Inventory,2021-07-07
2,1. Dashboard,2021-07-09


In [9]:
#### Step 3. Load Private Data Asset Inventory

# Asset Inventory (Private Access)
# https://data.cityofnewyork.us/dataset/Asset-Inventory/kvci-ugf9
# private_df = credentials.call_socrata_api('kvci-ugf9')
# private_df = private_df[['uid','audience','derived_view','parent_uid']]
# private_df.shape

In [10]:
# public_df = public_df.merge(private_df,on='uid',how='left')
# public_df.drop(columns=['audience'],inplace=True)
# public_df.shape

In [11]:
public_df.groupby(['type'])['row_count'].sum().round().sort_index().apply(lambda x: '%.f' % x)

type
datalens             0
dataset     3514217494
file                 0
filter       118236232
href                 0
map             622073
Name: row_count, dtype: object

In [12]:
# Create merged_filter, the dataframe that has only assets defined as datasets
# ZF approved the list
dataset_filter_list = ['dataset','filter','map']
public_filtered_df = public_df[public_df.type.isin(dataset_filter_list)]
public_filtered_df.shape

(2887, 12)

In [13]:
## remove derived assets if parent asset is public

# get parent ids for derived assets
parent_uids = public_filtered_df[public_filtered_df['derived_view']==True]['parent_uid']
# get ids for the assets derived from public assets
exc_parent_uids = public_filtered_df[public_filtered_df['uid'].isin(parent_uids)]['uid']
# remove derived assets if parent asset is public
public_filtered_df = public_filtered_df[~public_filtered_df['parent_uid'].isin(exc_parent_uids)]
public_filtered_df.shape

(2786, 12)

In [14]:
#### Step 4. Create one main dataset-level dataframe

# fix one date typo
public_filtered_df.loc[public_filtered_df['update_datemadepublic']=='August 9, 2-019',\
                       'update_datemadepublic'] = 'August 9, 2019'

# convert to date
public_filtered_df['update_datemadepublic'] = pd.to_datetime(
                                            pd.to_datetime(public_filtered_df['update_datemadepublic'],
                                                           errors='coerce')\
                                            .dt.strftime('%m/%d/%Y'), format=('%m/%d/%Y'))
public_filtered_df['last_data_updated_date'] = pd.to_datetime(
                                                 pd.to_datetime(public_filtered_df['last_data_updated_date'])\
                                                 .dt.strftime('%m/%d/%Y'))

# if agency is missing, create NA category
public_filtered_df['datasetinformation_agency'] = public_filtered_df['datasetinformation_agency'].fillna('Not filled out')
public_filtered_df.loc[public_filtered_df['datasetinformation_agency']=='','datasetinformation_agency'] = 'Not filled out'

keep_quant_cols=[
 'uid',
 'datasetinformation_agency',
 'name',
 'url',
 'type',
 'update_datemadepublic',
 'last_data_updated_date',
 'row_count'
]

quantity_dataset_df = public_filtered_df[keep_quant_cols]
quantity_dataset_df.shape

(2786, 8)

In [15]:
#### Step 5. Create one main agency-level dataframe

quantity_agency_df = quantity_dataset_df.groupby(['datasetinformation_agency'])\
                            .agg({'uid':'size','row_count':'sum'})\
                            .reset_index()\
                            .rename(columns={'uid':'numdatasets'})
quantity_agency_df.shape

(90, 3)

In [16]:
#### QUALITY (Data Freshness) ####

#### Step 1. Build baseline dataset

freshness_df = public_filtered_df[[
    'datasetinformation_agency',
    'name',
    'uid',
    'update_updatefrequency',
    'url',
    'update_datemadepublic',
    'last_data_updated_date',
    'update_automation']]

update_freq = ['Daily', 'Annually', 'Biannually ',
       'Quarterly', 'Monthly', 'Weekly', '2 to 4 times per year',
       'Weekdays', 'Every four years', 'Biweekly ', 'Triannually',
       'Several times per day', 'Hourly']

# Remove datasets with update frequencies for which we cannot determine freshness
freshness_df = freshness_df[(freshness_df['update_updatefrequency'].isin(update_freq)) &\
                             ~freshness_df['update_updatefrequency'].isna()]\
                            .reset_index(drop=True)

In [17]:
def assign_dataframe_statuses(data):

    """
    Determines if the data has been updated on time
    """
    
    df = data.copy()

    # some values have spaces
    df['update_updatefrequency'] = df['update_updatefrequency'].str.strip()
    
    # assign time by update frequency
    status_conditions = [
        (df['update_updatefrequency']=='Annually'),
        (df['update_updatefrequency']=='Monthly'),
        (df['update_updatefrequency']=='Quarterly'),
        (df['update_updatefrequency']=='Daily'),
        (df['update_updatefrequency']=='Biannually'),
        (df['update_updatefrequency']=='Weekly'),
        (df['update_updatefrequency']=='Triannually'),
        (df['update_updatefrequency']=='Weekdays'),
        (df['update_updatefrequency']=='2 to 4 times per year'),
        (df['update_updatefrequency']=='Biweekly'),
        (df['update_updatefrequency']=='Several times per day'),
        (df['update_updatefrequency']=='Hourly'),
        (df['update_updatefrequency']=='Every four years')
    ]
    status_choices = [
        pd.Timedelta('365 days'),
        pd.Timedelta('31 days'),
        pd.Timedelta('92 days'),
        pd.Timedelta('25 hours'),
        pd.Timedelta('182 days'),
        pd.Timedelta('7 days'),
        pd.Timedelta('122 days'),
        pd.Timedelta('5 days'),
        pd.Timedelta('182 days'),
        pd.Timedelta('4 days'),
        pd.Timedelta('25 hours'),
        pd.Timedelta('25 hours'),
        pd.Timedelta('1460 days')
        ]
    
    df['update_threshold'] = np.select(status_conditions, status_choices, default=pd.Timedelta('50000 days'))
    
    # calculate when asset should have been last updated
    df['last_updated_ago'] = pd.to_datetime(date.today()) - df['last_data_updated_date']
    
    # assign status to automated, dictionary and geocoded columns
    df['fresh'] = np.where((df['last_updated_ago']>=df['update_threshold']),'No','Yes')
    
    df.drop(columns=['update_threshold'],inplace=True)
    
    return df

freshness_df = assign_dataframe_statuses(freshness_df)

keep_fresh_cols = [
 'uid',
 'datasetinformation_agency',
 'name',
 'url',
 'update_automation',
 'update_updatefrequency',
 'last_data_updated_date',
 'fresh'    
]

freshness_dataset_df = freshness_df[keep_fresh_cols]
freshness_dataset_df.shape

(1140, 8)

In [18]:
freshness_dataset_df[freshness_dataset_df['uid']=='b4mf-rg6h']

,uid,datasetinformation_agency,name,url,update_automation,update_updatefrequency,last_data_updated_date,fresh
991,b4mf-rg6h,Department of Transportation (DOT),Commissioner's Correspondence,https://data.cityofnewyork.us/d/b4mf-rg6h,Yes,Biweekly,2021-07-02,No


In [19]:
list(freshness_dataset_df)

['uid',
 'datasetinformation_agency',
 'name',
 'url',
 'update_automation',
 'update_updatefrequency',
 'last_data_updated_date',
 'fresh']

In [20]:
update_values_used = {
    'Annually',
    'Monthly',
    'Quarterly',
    'Daily',
    'Biannually',
    'Weekly',
    'Triannually',
    'Weekdays',
    '2 to 4 times per year',
    'Biweekly',
    'Several times per day',
    'Hourly',
    'Every four years'
}

update_values_available = set(freshness_dataset_df['update_updatefrequency'].unique())
# identify new update frequency values
update_values_available.difference(update_values_used)

set()

In [21]:
#### Step 2. Calculate average data freshness by agency

# get the count of fresh dataset by agency
fresh_count_df = freshness_df[freshness_df['fresh']=='Yes'].groupby(['datasetinformation_agency'])\
                                .size()\
                                .reset_index()\
                                .rename(columns={0:'fresh_count'})

# get the total count of datasets by agency (excluding historical and as needed)
freshness_agency_df = freshness_df.groupby(['datasetinformation_agency'])\
                                .size()\
                                .reset_index()\
                                .rename(columns={0:'total_auto_count'})\
                                .merge(fresh_count_df, on='datasetinformation_agency',how='left')

# calculate percent freshly updated
freshness_agency_df['fresh_pct'] = freshness_agency_df['fresh_count'].fillna(0) / freshness_agency_df['total_auto_count']
freshness_agency_df.shape

(73, 4)

In [22]:
freshness_agency_df.head()

,datasetinformation_agency,total_auto_count,fresh_count,fresh_pct
0,311,5,4.0,0.800000
1,Administration for Children's Services (ACS),1,1.0,1.000000
2,Board of Standards and Appeals (BSA),1,NaN,0.000000
3,Brooklyn Borough President (BPBK),14,5.0,0.357143
4,Business Integrity Commission (BIC),9,9.0,1.000000


In [23]:
freshness_df[freshness_df['datasetinformation_agency']=='']

,datasetinformation_agency,name,uid,update_updatefrequency,url,update_datemadepublic,last_data_updated_date,update_automation,last_updated_ago,fresh


In [24]:
#### COMPLIANCE ####

#### Step 1. Build baseline dataset

# NYC Open Data Release Tracker
# https://data.cityofnewyork.us/City-Government/NYC-Open-Data-Release-Tracker/qj2z-ibhs
tracker_df = call_socrata_api('qj2z-ibhs')

# exclude Removed from the plan and Removed from the portal, 
release_status_filter = [
    'Released',
    'Scheduled for release',
    'Under Review'
]
tracker_df = tracker_df[tracker_df['release_status'].isin(release_status_filter)]

# apply grace period for release date
grace_period_days = 14
today = date.today()

# convert dates to dates
tracker_df['original_plan_date'] = pd.to_datetime(tracker_df['original_plan_date'])
tracker_df['latest_plan_date'] = pd.to_datetime(tracker_df['latest_plan_date'])
tracker_df['release_date'] = pd.to_datetime(tracker_df['release_date'])

# number of days between release and planned date
tracker_df['plan_to_release'] = (tracker_df['release_date'] - tracker_df['latest_plan_date']).dt.days

# create a check if released on time
tracker_df['within_grace_period'] = np.where((tracker_df['plan_to_release'] < grace_period_days), 'Yes', 'No')
tracker_df['within_grace_period_num'] = tracker_df['plan_to_release'] < grace_period_days

# subset datasets that were supposed to be released in the last 12 months
tracker_df['last_12_months'] = ((pd.to_datetime(today) - tracker_df['latest_plan_date']).dt.days < 365) & \
                                (tracker_df['latest_plan_date'] <= pd.to_datetime(today))

tracker_df['url'] = tracker_df['url1'].apply(lambda x: list(x.values())[0] \
                                                   if type(x) is dict else 'NA')
tracker_df.drop(columns=['url1'],inplace=True)
# drop duplicates for released datasets
# keep the one with the oldest release date
tracker_df = tracker_df[~tracker_df.u_id.isna()]\
                                .sort_values(by='release_date')\
                                .drop_duplicates(subset=['u_id'], keep='first')\
                                .append(tracker_df[tracker_df['u_id'].isna()])

tracker_12mo_df = tracker_df[tracker_df['last_12_months']]

tracker_12mo_df['latest_plan_date'] = tracker_12mo_df['latest_plan_date'].dt.strftime("%Y-%m-%d")
tracker_12mo_df['release_date'] = tracker_12mo_df['release_date'].dt.strftime("%Y-%m-%d")
tracker_12mo_df.shape

(173, 16)

In [25]:
tracker_12mo_df.head()

,agency,dataset,dataset_description,update_frequency,original_plan_date,latest_plan_date,release_status,agency_notes,from_the_2020_open_data_plan_,release_date,u_id,plan_to_release,within_grace_period,within_grace_period_num,last_12_months,url
949,Mayor's Office of Data Analytics (MODA),LL18 of 2019 Annual Pay Equity Report Dataset,Tables aggregating pay data for NYC employees ...,Annually,2020-10-31,2020-10-31,Released,NaN,Y,2020-01-15,423i-ukqr,-290.0,Yes,True,True,https://data.cityofnewyork.us/City-Government/...
851,Department of Transportation (DOT),Bollards Tracking and Installations,Database that tracks bollards installed in-house,To Be Determined,2019-12-31,2020-12-31,Released,NaN,NaN,2020-06-10,3f5t-9dqu,-204.0,Yes,True,True,https://data.cityofnewyork.us/Transportation/T...
582,Department of Social Services (DSS) - Departme...,Local Law 19 of 1999 Quarterly Reporting,"Data related to Street Outreach, Shelter place...",Quarterly,2020-11-30,2020-11-30,Released,NaN,Y,2020-09-15,7tu6-bcih,-76.0,Yes,True,True,https://data.cityofnewyork.us/Social-Services/...
1482,Office of Management and Budget (OMB),Contract Budget by Category,Citywide Contract Budget summarized by major c...,Annually,2020-12-31,2020-12-31,Released,NaN,Y,2020-09-23,h59m-jnyu,-99.0,Yes,True,True,https://data.cityofnewyork.us/City-Government/...
829,Office of Management and Budget (OMB),Projected Citywide Energy Costs,Citywide summary of projected energy costs,Biannually,2020-12-31,2020-12-31,Released,NaN,Y,2020-09-23,tyv9-j3ti,-99.0,Yes,True,True,https://data.cityofnewyork.us/City-Government/...


In [26]:
#### Step 2. Build dataset-level dataset

keep_tracker_cols = [
 'u_id',
 'agency',
 'dataset',
 'dataset_description',
 'latest_plan_date',
 'release_status',
 'release_date',
 'within_grace_period',
 'within_grace_period_num',
 'url'
]

tracker_12mo_dataset_df = tracker_12mo_df[keep_tracker_cols]

# append type and agency from public inventory
tracker_12mo_dataset_df = tracker_12mo_dataset_df.merge(public_df[['uid','type','datasetinformation_agency']], 
                                                                    left_on='u_id',
                                                                    right_on='uid',
                                                                    how='left')

# update agency name to match public inventory (can only be done for already published datasets)
tracker_12mo_dataset_df['datasetinformation_agency'] = np.where((tracker_12mo_dataset_df.release_status=='Released') & \
                                                                ~tracker_12mo_dataset_df['datasetinformation_agency'].isna(),
                                                                tracker_12mo_dataset_df['datasetinformation_agency'],
                                                                tracker_12mo_dataset_df['agency'])

# exclude assets that are not datasets, filters and gis maps
# keeps assets scheduled for release with type NA

tracker_12mo_dataset_df = tracker_12mo_dataset_df[tracker_12mo_dataset_df['u_id'].isin(quantity_dataset_df['uid']) | \
                                                  (tracker_12mo_dataset_df['release_status']=='Scheduled for release')]
tracker_12mo_dataset_df.drop(columns=['u_id','agency'],inplace=True)
tracker_12mo_dataset_df.shape

(169, 11)

In [27]:
tracker_12mo_dataset_df.head()

,dataset,dataset_description,latest_plan_date,release_status,release_date,within_grace_period,within_grace_period_num,url,uid,type,datasetinformation_agency
0,LL18 of 2019 Annual Pay Equity Report Dataset,Tables aggregating pay data for NYC employees ...,2020-10-31,Released,2020-01-15,Yes,True,https://data.cityofnewyork.us/City-Government/...,423i-ukqr,dataset,Mayor's Office of Data Analytics (MODA)
1,Bollards Tracking and Installations,Database that tracks bollards installed in-house,2020-12-31,Released,2020-06-10,Yes,True,https://data.cityofnewyork.us/Transportation/T...,3f5t-9dqu,dataset,Department of Transportation (DOT)
2,Local Law 19 of 1999 Quarterly Reporting,"Data related to Street Outreach, Shelter place...",2020-11-30,Released,2020-09-15,Yes,True,https://data.cityofnewyork.us/Social-Services/...,7tu6-bcih,dataset,Human Resources Administration (HRA)
3,Contract Budget by Category,Citywide Contract Budget summarized by major c...,2020-12-31,Released,2020-09-23,Yes,True,https://data.cityofnewyork.us/City-Government/...,h59m-jnyu,dataset,Mayor's Office of Management & Budget (OMB)
4,Projected Citywide Energy Costs,Citywide summary of projected energy costs,2020-12-31,Released,2020-09-23,Yes,True,https://data.cityofnewyork.us/City-Government/...,tyv9-j3ti,dataset,Mayor's Office of Management & Budget (OMB)


In [28]:
#### Step 3. Build agency-level dataset

# count number of overdue for release datasets
agency_overdue_df = tracker_12mo_dataset_df[tracker_12mo_dataset_df['release_status']=='Scheduled for release']\
                                .groupby(['datasetinformation_agency']).size().reset_index()\
                                .rename(columns={0:'overdue_datasets'})

tracker_12mo_agency_df = tracker_12mo_dataset_df.groupby(['datasetinformation_agency'])\
                                        .agg({'datasetinformation_agency':'size',
                                              'within_grace_period_num':'sum'})\
                                        .rename(columns={'datasetinformation_agency':'tracker_dataset_count',
                                                         'within_grace_period_num':'tracker_count_ontime'})\
                                        .merge(agency_overdue_df, on='datasetinformation_agency', how='left')\
                                        .reset_index(drop=True)\
                                        .fillna(0)

# calculate percent released on time
tracker_12mo_agency_df['pct_ontime'] = tracker_12mo_agency_df['tracker_count_ontime'].fillna(0)/tracker_12mo_agency_df['tracker_dataset_count']
tracker_12mo_agency_df.shape

(42, 5)

In [29]:
tracker_12mo_agency_df.head()

,datasetinformation_agency,tracker_dataset_count,tracker_count_ontime,overdue_datasets,pct_ontime
0,Administration for Children's Services (ACS),1,1,0.0,1.0
1,Brooklyn Borough President's Office,13,0,13.0,0.0
2,Campaign Finance Board (CFB),10,0,0.0,0.0
3,Commission on Human Rights (CCHR),1,0,0.0,0.0
4,Department of Buildings (DOB),2,1,1.0,0.5


In [46]:
#### DASHBOARD ####

#### Step 1. Get citywide metrics

# total number of rows
cw_numrows = quantity_agency_df['row_count'].sum()

# total number of datasets
cw_numdatasets = quantity_agency_df['numdatasets'].sum()
# percent updated on time
cw_freshness = freshness_dataset_df[freshness_dataset_df['fresh']=='Yes'].shape[0]/\
                    freshness_df.shape[0]
# percent released on time
cw_compliance = tracker_12mo_dataset_df['within_grace_period_num'].sum()/ \
                tracker_12mo_dataset_df.shape[0]

# number of assets that were supposed to be released but were not as of today 
cw_overdue = tracker_12mo_dataset_df[tracker_12mo_dataset_df['release_status']=='Scheduled for release'].shape[0]

citywide_df = pd.DataFrame([['Citywide',
                         cw_numrows,
                         cw_numdatasets,
                         cw_freshness,
                         cw_compliance,
                         cw_overdue]],
                       columns=['Scope',
                                'Number of published rows',
                                'Number of published datasets',
                                'Percent of datasets updated on time',
                                'Percent of datasets released on time in the last 12 months',
                                'Number of overdue datasets'])

citywide_df

,Scope,Number of published rows,Number of published datasets,Percent of datasets updated on time,Percent of datasets released on time in the last 12 months,Number of overdue datasets
0,Citywide,3.630447e+09,2786,0.716667,0.325444,85


In [31]:
#### Step 2. Build complete agency-level dataset

all_agency_df = quantity_agency_df.merge(freshness_agency_df, 
                                        on='datasetinformation_agency',
                                        how='outer')\
                                  .merge(tracker_12mo_agency_df, 
                                        on='datasetinformation_agency',
                                        how='outer')

all_agency_df['overdue_datasets'] = all_agency_df['overdue_datasets'].fillna(0)
all_agency_df['numdatasets'] = all_agency_df['numdatasets'].fillna(0)
all_agency_df['numrows'] = all_agency_df['row_count'].fillna(0)
all_agency_df['total_auto_count'] = all_agency_df['total_auto_count'].fillna(0)
all_agency_df['fresh_count'] = all_agency_df['fresh_count'].fillna(0)
all_agency_df['tracker_dataset_count'] = all_agency_df['tracker_dataset_count'].fillna(0)
all_agency_df['tracker_count_ontime'] = all_agency_df['tracker_count_ontime'].fillna(0)
all_agency_df['fresh_pct'] = all_agency_df['fresh_pct'].fillna('No automated datasets')
all_agency_df['pct_ontime'] = all_agency_df['pct_ontime'].fillna('No datasets in the tracker')
all_agency_df.shape

(99, 11)

In [32]:
all_agency_df = all_agency_df[[
                'datasetinformation_agency',
                'numdatasets',
                'numrows',
                'fresh_pct',
                'pct_ontime',
                'overdue_datasets'    
]]

all_agency_df.rename(columns={
                'datasetinformation_agency':'Agency',
                'numdatasets':'Number of datasets',
                'numrows':'Number of rows',
                'fresh_pct':'Percent of datasets updated on time',
                'pct_ontime':'Percent of planned releases released on time within last 12 months',
                'overdue_datasets':'Number of overdue for release datasets'
}, inplace=True)

In [33]:
# all_agency_df.rename(columns={'datasetinformation_agency':'agency'},inplace=True)
# all_agency_df = all_agency_df[[
#                 'agency',
#                 'numdatasets',
#                 'numrows',
#                 'total_auto_count',
#                 'fresh_count',
#                 'fresh_pct',
#                 'tracker_dataset_count',
#                 'tracker_count_ontime',
#                 'overdue_datasets',
#                 'pct_ontime'    
# ]]

In [34]:
#### Step 3. Build complete dataset-level dataset

# aggregate freshness data and tracker data (for released datasets only)
all_datasets_df = quantity_dataset_df.merge(freshness_dataset_df[['uid',
                                                                  'update_automation',
                                                                  'update_updatefrequency',
                                                                  'fresh']], 
                                        on='uid',
                                        how='outer')\
                                  .merge(tracker_12mo_dataset_df[['uid',
                                                                  'dataset_description',
                                                                  'latest_plan_date',   
                                                                  'release_status',
                                                                  'release_date',
                                                                  'within_grace_period',
                                                                  'within_grace_period_num']], 
                                        on='uid',
                                        how='left')
all_datasets_df.shape

(2786, 17)

In [35]:
tracker_12mo_dataset_df.head()

,dataset,dataset_description,latest_plan_date,release_status,release_date,within_grace_period,within_grace_period_num,url,uid,type,datasetinformation_agency
0,LL18 of 2019 Annual Pay Equity Report Dataset,Tables aggregating pay data for NYC employees ...,2020-10-31,Released,2020-01-15,Yes,True,https://data.cityofnewyork.us/City-Government/...,423i-ukqr,dataset,Mayor's Office of Data Analytics (MODA)
1,Bollards Tracking and Installations,Database that tracks bollards installed in-house,2020-12-31,Released,2020-06-10,Yes,True,https://data.cityofnewyork.us/Transportation/T...,3f5t-9dqu,dataset,Department of Transportation (DOT)
2,Local Law 19 of 1999 Quarterly Reporting,"Data related to Street Outreach, Shelter place...",2020-11-30,Released,2020-09-15,Yes,True,https://data.cityofnewyork.us/Social-Services/...,7tu6-bcih,dataset,Human Resources Administration (HRA)
3,Contract Budget by Category,Citywide Contract Budget summarized by major c...,2020-12-31,Released,2020-09-23,Yes,True,https://data.cityofnewyork.us/City-Government/...,h59m-jnyu,dataset,Mayor's Office of Management & Budget (OMB)
4,Projected Citywide Energy Costs,Citywide summary of projected energy costs,2020-12-31,Released,2020-09-23,Yes,True,https://data.cityofnewyork.us/City-Government/...,tyv9-j3ti,dataset,Mayor's Office of Management & Budget (OMB)


In [36]:
# append non-released datasets data
# doing it as a separate step to keep more accurate data for released datasets
all_datasets_df = all_datasets_df.append(tracker_12mo_dataset_df[~tracker_12mo_dataset_df['uid'].isin(all_datasets_df['uid'])])\
                                 .reset_index(drop=True)

# merge name and dataset columns since they contain the same information
all_datasets_df.loc[all_datasets_df.name.isna(),'name'] = all_datasets_df['dataset']

# merge automation/update data for "historical" and "as needed" datasets
all_datasets_df = all_datasets_df.merge(public_df[['uid','update_automation','update_updatefrequency']], on='uid', how='left')

all_datasets_df['automation'] = np.where(all_datasets_df['update_automation_x'].isna(),
                                         all_datasets_df['update_automation_y'],
                                         all_datasets_df['update_automation_x'])
all_datasets_df['update_frequency'] = np.where(all_datasets_df['update_updatefrequency_x'].isna(),
                                         all_datasets_df['update_updatefrequency_y'],
                                         all_datasets_df['update_updatefrequency_x'])
# recode missing dates into string NA to properly read format in GDS
all_datasets_df['release_date_fix'] = pd.to_datetime(all_datasets_df['release_date'], errors='coerce')

all_datasets_df.shape

(2871, 23)

In [37]:
all_datasets_df.head()

,uid,datasetinformation_agency,name,url,type,update_datemadepublic,last_data_updated_date,row_count,update_automation_x,update_updatefrequency_x,...,release_status,release_date,within_grace_period,within_grace_period_num,dataset,update_automation_y,update_updatefrequency_y,automation,update_frequency,release_date_fix
0,dq7m-xn5i,Department of Transportation (DOT),VZV_Hands-On Safety Demos,https://data.cityofnewyork.us/d/dq7m-xn5i,map,2018-05-31,2021-06-28,NaN,Yes,Monthly,...,NaN,NaN,NaN,NaN,NaN,Yes,Monthly,Yes,Monthly,NaT
1,5jsj-cq4s,Department of Transportation (DOT),Parking Meters GPS Coordinates and Status,https://data.cityofnewyork.us/d/5jsj-cq4s,map,2017-10-23,2021-06-28,NaN,Yes,Monthly,...,NaN,NaN,NaN,NaN,NaN,Yes,Monthly,Yes,Monthly,NaT
2,g5vn-nwqt,Department of Transportation (DOT),VZV_Signal Timing/25MPH Signal Retiming,https://data.cityofnewyork.us/d/g5vn-nwqt,map,2018-05-31,2021-06-30,NaN,Yes,Monthly,...,NaN,NaN,NaN,NaN,NaN,Yes,Monthly,Yes,Monthly,NaT
3,bheb-sjfi,Department of Transportation (DOT),Sidewalk Correspondences,https://data.cityofnewyork.us/d/bheb-sjfi,dataset,2019-11-20,2021-06-30,17717.0,No,Quarterly,...,NaN,NaN,NaN,NaN,NaN,No,Quarterly,No,Quarterly,NaT
4,sqcr-6mww,311,311 Service Requests for 2004,https://data.cityofnewyork.us/d/sqcr-6mww,dataset,2011-10-20,2011-10-20,1118286.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,No,Historical Data,No,Historical Data,NaT


In [38]:
all_datasets_df.loc[all_datasets_df['update_frequency'].isin(['Historical Data','As needed']),'fresh'] = 'No regular updates'
all_datasets_df.loc[all_datasets_df['release_status']=='Scheduled for release','fresh'] = 'Not yet released'
all_datasets_df['fresh'] = all_datasets_df['fresh'].fillna('Not determined')
# all_datasets_df['fresh'] = all_datasets_df['fresh'].fillna('No regular updates')
all_datasets_df['within_grace_period'] = all_datasets_df['within_grace_period'].fillna('Not in Open Plan Tracker')
all_datasets_df.shape

(2871, 23)

In [39]:
all_datasets_df= all_datasets_df[[
    'datasetinformation_agency',
    'name',
    'url',
    'type',
    'update_datemadepublic',
    'last_data_updated_date',
    'automation',
    'update_frequency',
    'fresh',
    'latest_plan_date',
    'release_date_fix',
    'within_grace_period',
    'row_count',
    'release_status',
    'dataset_description'
    ]]

all_datasets_df.rename(columns={
    'datasetinformation_agency':'Agency',
    'name':'Dataset name',
    'url':'URL',
    'type':'Asset type',
    'update_datemadepublic':'Date made public',
    'last_data_updated_date':'Last updated on',
    'automation':'Automation',
    'update_frequency':'Update frequency',
    'fresh':'Updated on time',
    'latest_plan_date':'Latest Open Data Plan release date',
    'release_date_fix':'Open Data Plan release date',
    'within_grace_period':'Planned releases released on  time within last 12mo?',
    'row_count':'Number of rows',
    'release_status':'Open Data Plan Release Status',
    'dataset_description':'Description'
},inplace=True)

In [40]:
all_datasets_df['Updated on time'].value_counts(dropna=False)

No regular updates    1637
Yes                    817
No                     323
Not yet released        85
Not determined           9
Name: Updated on time, dtype: int64

In [41]:
all_datasets_df[all_datasets_df['Updated on time']=='Not determined']['Update frequency'].value_counts(dropna=False)

To Be Determined    4
Bimonthly           2
Every six months    2
Biennially          1
Name: Update frequency, dtype: int64

In [42]:
all_datasets_df[all_datasets_df['Update frequency'].isin(['Biennially','Bimonthly '])]

,Agency,Dataset name,URL,Asset type,Date made public,Last updated on,Automation,Update frequency,Updated on time,Latest Open Data Plan release date,Open Data Plan release date,Planned releases released on time within last 12mo?,Number of rows,Open Data Plan Release Status,Description
1215,Mayor's Office of Management & Budget (OMB),Debt Service Major Indicators,https://data.cityofnewyork.us/d/4yay-s2us,dataset,2020-11-25,2021-05-04,No,Bimonthly,Not determined,2020-12-31,2020-11-25,Yes,39.0,Released,Annual budgeted expense by unit of appropriation
1407,Department of Citywide Administrative Services...,EEO-4 Reports,https://data.cityofnewyork.us/d/dbpt-pbmd,dataset,2020-12-09,2020-12-07,No,Biennially,Not determined,2020-11-30,2020-12-09,Yes,537.0,Released,A federally-mandated report that provides a su...
1840,Department of City Planning (DCP),Primary Land Use Tax Lot Output (PLUTO),https://data.cityofnewyork.us/d/64uk-42ks,dataset,2020-02-12,2021-06-30,No,Bimonthly,Not determined,NaN,NaT,Not in Open Plan Tracker,858213.0,NaN,NaN


In [43]:
# all_datasets_df.rename(columns={
#  'datasetinformation_agency':'agency',
#  'uid':'u_id',
#  'url':'dataset_link',
#  'update_datemadepublic':'date_made_public_dt',
#  'row_count':'numrows',
#  'last_data_updated_date':'last_update_date_data_dt',
#  'latest_plan_date':'latest_plan_date_dt',
#  'release_date_fix':'release_date_dt_fix'
# },inplace=True)

# all_datasets_df= all_datasets_df[[
#     'agency',
#     'u_id',
#     'name',
#     'dataset_description',
#     'dataset_link',
#     'type',
#     'date_made_public_dt',
#     'numrows',
#     'automation',
#     'update_frequency',
#     'last_update_date_data_dt',
#     'fresh',
#     'latest_plan_date_dt',
#     'release_status',
#     'release_date_dt_fix',
#     'within_grace_period'
# ]]

In [44]:
all_datasets_df.head()

,Agency,Dataset name,URL,Asset type,Date made public,Last updated on,Automation,Update frequency,Updated on time,Latest Open Data Plan release date,Open Data Plan release date,Planned releases released on time within last 12mo?,Number of rows,Open Data Plan Release Status,Description
0,Department of Transportation (DOT),VZV_Hands-On Safety Demos,https://data.cityofnewyork.us/d/dq7m-xn5i,map,2018-05-31,2021-06-28,Yes,Monthly,Yes,NaN,NaT,Not in Open Plan Tracker,NaN,NaN,NaN
1,Department of Transportation (DOT),Parking Meters GPS Coordinates and Status,https://data.cityofnewyork.us/d/5jsj-cq4s,map,2017-10-23,2021-06-28,Yes,Monthly,Yes,NaN,NaT,Not in Open Plan Tracker,NaN,NaN,NaN
2,Department of Transportation (DOT),VZV_Signal Timing/25MPH Signal Retiming,https://data.cityofnewyork.us/d/g5vn-nwqt,map,2018-05-31,2021-06-30,Yes,Monthly,Yes,NaN,NaT,Not in Open Plan Tracker,NaN,NaN,NaN
3,Department of Transportation (DOT),Sidewalk Correspondences,https://data.cityofnewyork.us/d/bheb-sjfi,dataset,2019-11-20,2021-06-30,No,Quarterly,Yes,NaN,NaT,Not in Open Plan Tracker,17717.0,NaN,NaN
4,311,311 Service Requests for 2004,https://data.cityofnewyork.us/d/sqcr-6mww,dataset,2011-10-20,2011-10-20,No,Historical Data,No regular updates,NaN,NaT,Not in Open Plan Tracker,1118286.0,NaN,NaN


In [51]:
all_datasets_df.dtypes

Agency                                                          object
Dataset name                                                    object
URL                                                             object
Asset type                                                      object
Date made public                                        datetime64[ns]
Last updated on                                         datetime64[ns]
Automation                                                      object
Update frequency                                                object
Updated on time                                                 object
Latest Open Data Plan release date                              object
Open Data Plan release date                             datetime64[ns]
Planned releases released on  time within last 12mo?            object
Number of rows                                                 float64
Open Data Plan Release Status                                   object
Descri

In [61]:
list(all_datasets_df)

['Agency',
 'Dataset name',
 'URL',
 'Asset type',
 'Date made public',
 'Last updated on',
 'Automation',
 'Update frequency',
 'Updated on time',
 'Latest Open Data Plan release date',
 'Open Data Plan release date',
 'Planned releases released on  time within last 12mo?',
 'Number of rows',
 'Open Data Plan Release Status',
 'Description']

In [63]:
all_datasets_df['Planned releases released on  time within last 12mo?'].value_counts(dropna=True)

Not in Open Plan Tracker    2702
No                           114
Yes                           55
Name: Planned releases released on  time within last 12mo?, dtype: int64

In [56]:
all_datasets_df['Date made public'] = all_datasets_df['Date made public'].dt.strftime('%Y-%m-%d')
all_datasets_df['Last updated on'] = all_datasets_df['Last updated on'].dt.strftime('%Y-%m-%d')
all_datasets_df['Open Data Plan release date'] = all_datasets_df['Open Data Plan release date'].dt.strftime('%Y-%m-%d')

In [57]:
all_datasets_df.head()

,Agency,Dataset name,URL,Asset type,Date made public,Last updated on,Automation,Update frequency,Updated on time,Latest Open Data Plan release date,Open Data Plan release date,Planned releases released on time within last 12mo?,Number of rows,Open Data Plan Release Status,Description
0,Department of Transportation (DOT),VZV_Hands-On Safety Demos,https://data.cityofnewyork.us/d/dq7m-xn5i,map,2018-05-31,2021-06-28,Yes,Monthly,Yes,NaN,NaN,Not in Open Plan Tracker,NaN,NaN,NaN
1,Department of Transportation (DOT),Parking Meters GPS Coordinates and Status,https://data.cityofnewyork.us/d/5jsj-cq4s,map,2017-10-23,2021-06-28,Yes,Monthly,Yes,NaN,NaN,Not in Open Plan Tracker,NaN,NaN,NaN
2,Department of Transportation (DOT),VZV_Signal Timing/25MPH Signal Retiming,https://data.cityofnewyork.us/d/g5vn-nwqt,map,2018-05-31,2021-06-30,Yes,Monthly,Yes,NaN,NaN,Not in Open Plan Tracker,NaN,NaN,NaN
3,Department of Transportation (DOT),Sidewalk Correspondences,https://data.cityofnewyork.us/d/bheb-sjfi,dataset,2019-11-20,2021-06-30,No,Quarterly,Yes,NaN,NaN,Not in Open Plan Tracker,17717.0,NaN,NaN
4,311,311 Service Requests for 2004,https://data.cityofnewyork.us/d/sqcr-6mww,dataset,2011-10-20,2011-10-20,No,Historical Data,No regular updates,NaN,NaN,Not in Open Plan Tracker,1118286.0,NaN,NaN


In [64]:
all_datasets_df.columns.values.tolist()

['Agency',
 'Dataset name',
 'URL',
 'Asset type',
 'Date made public',
 'Last updated on',
 'Automation',
 'Update frequency',
 'Updated on time',
 'Latest Open Data Plan release date',
 'Open Data Plan release date',
 'Planned releases released on  time within last 12mo?',
 'Number of rows',
 'Open Data Plan Release Status',
 'Description']

In [65]:
all_datasets_df.shape

(2871, 15)

In [71]:
all_datasets_df.tail(1).values.tolist()

[["Mayor's Office of Housing Recovery Operations",
  'Patterns of Attrition and Retention in the Build it Back Program',
  'NA',
  nan,
  nan,
  nan,
  nan,
  nan,
  'Not yet released',
  '2021-05-31',
  nan,
  'No',
  nan,
  'Scheduled for release',
  'Dataset analyzes patterns of homeowner-applicant attrition and retention in the Build it Back Program.']]

In [69]:
gs_upload(wks_name='_datasets_',
          df=all_datasets_df.tail(1))

APIError: {'code': 400, 'message': 'Invalid JSON payload received. Unexpected token.\nack Program", "NA", NaN, NaN, NaN, NaN, \n                    ^', 'status': 'INVALID_ARGUMENT'}

In [74]:
#### Step 4. Upload data to Google Spreadsheets

# gs_upload(wks_name='_citywide_',
#           df=citywide_df)
# print('Upload complete for citywide dataset')

# gs_upload(wks_name='_agency_',
#           df=all_agency_df)
# print('Upload complete for agency dataset')

# gs_upload(wks_name='_datasets_',
#           df=all_datasets_df)
# print('Upload complete for datasets dataset')

# gs_upload(wks_name='_datasets_not_released_',
#           df=not_released_datasets_df)
# print('Upload complete for not released datasets dataset')

# gs_upload(wks_name='_dates_',
#           df=dates_df)
# print('Upload complete for dates dataset')

# print(f"Dashboard was updated at: {datetime.now()}")

In [82]:
#### Step 4. Upload data to Google Spreadsheets

gs_upload(df=citywide, 
                    wks_name='_citywide_')
print('Upload complete for citywide dataset')

# credentials.gs_upload(df=all_agency_df, 
#                     wks_name='_agency_')
# print('Upload complete for agency dataset')

# credentials.gs_upload(df=all_datasets_df, 
#                     wks_name='_datasets_')
# print('Upload complete for datasets dataset')

# credentials.gs_upload(df=dates_df, 
#                     wks_name='_dates_')
# print('Upload complete for dates dataset')

Upload complete for citywide dataset


In [ ]:
print(f"Dashboard was updated on: {date.today()}")

In [ ]:
citywide